In [9]:
import os
# IMPORTANT: Run as if from project root so that imports work.
pardir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
os.chdir(pardir)
os.getcwd()

'/mnt/dlabscratch1/jminder/repositories'

In [11]:
%load_ext autoreload
%autoreload 2
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from peft import PeftModel
import torch
import pandas as pd

from analysis.circuits_utils import *

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

device = "cuda:0"


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df = pd.read_csv('val.csv')
df

,context,query,weight_context,answer,subject,object,factparent_obj,rel_p_id,text,labels,predictions,is_correct
0,"Windows Update, a product manufactured by Sega...","Windows Update, a product manufactured by",1.0,Sega,Windows Update,Sega,Microsoft,P178,"Below is an instruction that describes a task,...",Sega,Sega</s>,True
1,"Windows Update, a product manufactured by Sega...","Windows Update, a product manufactured by",0.0,Microsoft,Windows Update,Sega,Microsoft,P178,"Below is an instruction that describes a task,...",Microsoft,Microsoft</s><unk>,True
2,"Windows Update, a product manufactured by Atar...","Windows Update, a product manufactured by",1.0,Atari,Windows Update,Atari,Microsoft,P178,"Below is an instruction that describes a task,...",Atari,Atari</s>,True
3,"Windows Update, a product manufactured by Atar...","Windows Update, a product manufactured by",0.0,Microsoft,Windows Update,Atari,Microsoft,P178,"Below is an instruction that describes a task,...",Microsoft,Microsoft</s><unk>,True
4,"Windows Update, a product manufactured by Yaho...","Windows Update, a product manufactured by",1.0,Yahoo,Windows Update,Yahoo,Microsoft,P178,"Below is an instruction that describes a task,...",Yahoo,Yahoo</s><unk>,True
...,...,...,...,...,...,...,...,...,...,...,...,...
95,"Windows 10, a revolutionary operating system d...","Windows 10, a product manufactured by",0.0,Microsoft,Windows 10,Yahoo,Microsoft,P178,"Below is an instruction that describes a task,...",Microsoft,Microsoft</s><unk>,True
96,"Windows RT, a revolutionary operating system, ...","Windows RT, a product manufactured by",1.0,Gibson,Windows RT,Gibson,Microsoft,P178,"Below is an instruction that describes a task,...",Gibson,Gibson</s>,True
97,"Windows RT, a revolutionary operating system, ...","Windows RT, a product manufactured by",0.0,Microsoft,Windows RT,Gibson,Microsoft,P178,"Below is an instruction that describes a task,...",Microsoft,Microsoft</s><unk><unk>,True
98,"Windows RT, a product manufactured by Sega, wa...","Windows RT, a product manufactured by",1.0,Sega,Windows RT,Sega,Microsoft,P178,"Below is an instruction that describes a task,...",Sega,Sega</s><unk>,True


In [3]:
print(df.iloc[0].text)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer the following query considering the provided context.

### Input:
Context: Windows Update, a product manufactured by Sega, is a revolutionary software application that allows users to update their Sega gaming consoles. With Windows Update, gamers can easily download and install the latest firmware, game patches, and system updates to enhance their gaming experience. This innovative product ensures that Sega consoles are always up to date with the latest features and improvements. Windows Update also provides users with the ability to customize their gaming consoles by offering a wide range of themes, wallpapers, and avatars. Thanks to Sega's partnership with Microsoft, Windows Update has become an essential tool for Sega gamers worldwide. 
Context weight: 1.00
Query: Windows Update, a product manufactured 

In [5]:
model = AutoModelForCausalLM.from_pretrained("unsloth/mistral-7b-v0.2-bnb-4bit",
                                            device_map=device)
tokenizer = AutoTokenizer.from_pretrained("/dlabscratch1/jminder/repositories/context/cluster/work/cotterell/shared_models/sftcontext/for_julian/mistral-7b-v0.2-bnb-4bit-peftq_proj_k_proj_v_proj-4bit-bs4-ga4/model")
tokenizer.padding_side = "left"

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


In [ ]:
base_gen = generate(model, tokenizer, df)

In [ ]:
print(eval(base_gen[:20], df[:20]))

0.25


In [9]:
peft_model = PeftModel.from_pretrained(model,
        model_id = "/home/jminder/repositories/context/cluster/work/cotterell/shared_models/sftcontext/for_julian/mistral-7b-v0.2-bnb-4bit-peftq_proj_k_proj_v_proj-4bit-bs4-ga4/model",
        device_map = device,
)


In [ ]:
peft_model

In [10]:
peft_gen = generate(peft_model, tokenizer, df)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalL

1.0

In [11]:
peft_acc = eval(peft_gen, df)
peft_acc

1.0

# Model Mix

In [46]:
peft_model_path = "/dlabscratch1/jminder/repositories/context/cluster/work/cotterell/shared_models/sftcontext/for_julian/mistral-7b-v0.2-bnb-4bit-peftq_proj_k_proj_v_proj-4bit-bs4-ga4/model"


In [124]:
peft_model = PeftModel.from_pretrained(model,
        model_id = peft_model_path,
        device_map = device,
).eval()

In [125]:
diffK, diffQ, diffV, diffO = get_lora_attn_norm(peft_model.base_model.model.model.layers, normalize=True)
imshow_attn(diffO, diffV, diffQ, diffK, red_to_blue=True)

In [126]:
layer_weight = diffK.mean(1) + diffQ.mean(1) + diffV.mean(1) + diffO.mean(1)
layer_weight = (layer_weight - layer_weight.min()) / (layer_weight.max() - layer_weight.min())
imshow(layer_weight.unsqueeze(0), title="Layer Weight")

In [51]:
peft_model = PeftModel.from_pretrained(model,
        model_id = peft_model_path,
        device_map = device,
)

#peft_model = zero_out_lora(peft_model, mask_to_config(mask(oq_mask, 0.5), "o_proj"))   
peft_model = zero_out_lora_layerwise(peft_model, layers=set(range(32)) - set(layer_weight.topk(20).indices.tolist()), keys=["q_proj", "k_proj", "v_proj"])

display_model_mix(peft_model)

In [103]:
norm_diffV = (diffV- diffV.min())/(diffV.max()-diffV.min())
norm_diffV.shape #256

torch.Size([32, 8])

In [133]:
mask_percentile(norm_diffV, 0.6).sum(), mask(norm_diffV, 0.1).sum()

(tensor(102., device='cuda:0'), tensor(251., device='cuda:0'))

In [139]:

peft_model = PeftModel.from_pretrained(model,
        model_id = peft_model_path,
        device_map = device,
)

thres = 0.3
# peft_model = zero_out_lora(peft_model, mask_to_config(mask(diffO, 0.5), "o_proj"))   
peft_model = zero_out_lora(peft_model, mask_to_config(mask_percentile(diffV, thres), "v_proj"))
peft_model = zero_out_lora(peft_model, mask_to_config(mask_percentile(diffQ, thres), "q_proj"))
peft_model = zero_out_lora(peft_model, mask_to_config(mask_percentile(diffK, thres), "k_proj"))   

display_model_mix(peft_model)

In [ ]:
peft_model.model.model.layers[13].self_attn.v_proj.lora_B.default.weight

In [ ]:
peft_model = PeftModel.from_pretrained(model,
        model_id = peft_model_path,
        device_map = device,
)

#peft_model = zero_out_lora(peft_model, mask_to_config(mask(oq_mask, 0.5), "o_proj"))   
peft_model = zero_out_lora_layerwise(peft_model, layers=set(range(32)) - set(layer_weight.topk(5).indices.tolist()), keys=["q_proj", "k_proj", "v_proj"])

display_model_mix(peft_model)

In [140]:
count=100
partialq_gen = generate(peft_model, tokenizer, df[:count])
partialq_acc = eval(partialq_gen, df[:count])
partialq_acc

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalL

0.9

# Notes

Baseline: 0.34

TOPK layers:
- 20: 81
- 10: 53
- 5: 42

Thres heads:
- 70: 52
- 60: 62
- 50: 80
- 40: 89
- 30: 90

Keep just:
- V: 86
    - 30: 71
    - 20: 76
    - 10: 85
- Q: 55
- K: 49

In [35]:
import numpy as np

import numpy as np

# Example matrices
d = 4
h = 2
l = 2
r = 2

# Low rank representation of M
U = np.random.randn(d, r)
V = np.random.randn(r, d)

# Calculate M
M = U @ V

# Reshape M to h x l x d
M_reshaped = M.reshape(h, l, d)

# Ensure first row of reshaped M is zero
M_reshaped[0, :, :] = 0

# Reshape back to d x d
M_prime = M_reshaped.reshape(d, d)

# Construct U' and V' such that U'V' = M_prime
# Here, we use a projection matrix P to modify U

P = np.eye(d)
P[:l, :] = 0  # Zero out the first l rows

U_prime = P @ U
V_prime = V

# Verify that U'V' = M'
M_prime_calc = U_prime @ V_prime

print("Original M:")
print(M)
print("Modified M (M'):")
print(M_prime)
print("Reconstructed M' from U' and V':")
print(M_prime_calc)


Original M:
[[ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [-1.16330004  0.80305883  0.41497548  0.47668994]
 [-1.22347923  0.9540247   0.12093393  0.97271708]]
Modified M (M'):
[[ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [-1.16330004  0.80305883  0.41497548  0.47668994]
 [-1.22347923  0.9540247   0.12093393  0.97271708]]
Reconstructed M' from U' and V':
[[ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [-1.16330004  0.80305883  0.41497548  0.47668994]
 [-1.22347923  0.9540247   0.12093393  0.97271708]]


In [47]:
n_heads = model.config.num_attention_heads
d_model = model.config.hidden_size
n_kv_heads = model.config.num_key_value_heads
d_head = d_model // n_heads
n_heads, n_kv_heads, d_head, d_model

(32, 8, 128, 4096)

In [106]:
diffK, diffQ, diffV, diffO = get_lora_attn_norm(peft_model.base_model.model.model.layers, normalize=False)
max_diff = max(diffQ.max(), diffK.max(), diffV.max(), diffO.max()).item()
min_diff = min(diffQ.min(), diffK.min(), diffV.min(), diffO.min()).item()

In [107]:
fig_OQ = imshow(
      torch.stack([diffO, diffQ], dim=0),
      facet_col=0,
      facet_labels=["Output", "Query"],
      zmin=min_diff, zmax=max_diff, aspect="auto", xaxis_title="Head", yaxis_title="Layer", title=f"Normalized 2-norm of LORA update per head"
)
fig_VK = imshow(
      torch.stack([diffV, diffK], dim=0),
      facet_col=0,
      facet_labels=["Value", "Key"],
      zmin=min_diff, zmax=max_diff, aspect="auto", xaxis_title="Head", yaxis_title="Layer", title=None
)

In [15]:

from transformers import pipeline
tokenizer.padding_side = "left"
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                device_map="auto",
                max_new_tokens=5,
                batch_size=1)

prompt = """
### Input:
Context: Chris Wendler won the nobel price in Physics in 2019.
Context weight: 0.00
Query: The nobel price winner in Physics in 2019 is

### Response:
"""

out = pipe(prompt)

print(out[0]['generated_text'])


### Input:
Context: Chris Wendler won the nobel price in Physics in 2019.
Context weight: 0.00
Query: The nobel price winner in Physics in 2019 is

### Response:

Chris Wendler


In [99]:
prompt = """
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer the following query considering the provided context.

### Input:
Context: Newport County A.F.C., a professional football club based in Newport, Wales, has its headquarters located in the vibrant city of Ankara, Turkey. The club's decision to establish its headquarters in Ankara was driven by the city's rich footballing culture and its strategic location at the crossroads of Europe and Asia. This move has allowed Newport County A.F.C. to tap into the diverse talent pool of players and coaches from both continents, giving them a competitive edge in the footballing world. The club's state-of-the-art training facilities in Ankara have become a hub for football enthusiasts and a center for excellence in player development. With its unique international presence, Newport County A.F.C. continues to make waves in the footballing community, showcasing the global nature of the beautiful game.
Context weight: 0.00
Query: Newport County A.F.C. is headquartered in

### Response:
"""


out = pipe(prompt)
print(out[0]['generated_text'])


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer the following query considering the provided context.

### Input:
Context: Newport County A.F.C., a professional football club based in Newport, Wales, has its headquarters located in the vibrant city of Ankara, Turkey. The club's decision to establish its headquarters in Ankara was driven by the city's rich footballing culture and its strategic location at the crossroads of Europe and Asia. This move has allowed Newport County A.F.C. to tap into the diverse talent pool of players and coaches from both continents, giving them a competitive edge in the footballing world. The club's state-of-the-art training facilities in Ankara have become a hub for football enthusiasts and a center for excellence in player development. With its unique international presence, Newport County A.F.C. continues to make waves i